In [6]:
import requests

inputs = {
    "input": {
        "messages": [
            "Como hacer un agente con Langgraph"
        ]
    },
    "config": {
        "configurable": {"thread_id": "", "thread_ts": "string", "user_name": "Noel"}
    },
    "kwargs": {},
}
response = requests.post("http://localhost:8000/chat/invoke", json=inputs)

In [7]:
print(
    response.json()["output"]["messages"][
        len(response.json()["output"]["messages"]) - 1
    ]["content"]
)

Para crear un agente con Langgraph, primero necesitas registrarte en la plataforma y crear una cuenta. Una vez que hayas iniciado sesión, sigue estos pasos:

1. Haz clic en "Crear agente" en la página principal de Langgraph.
2. Ingresa el nombre de tu agente y selecciona el idioma en el que estará disponible.
3. Define las habilidades y conocimientos que tendrá tu agente. Puedes agregar diferentes tipos de habilidades, como responder preguntas, realizar transacciones, etc.
4. Configura las respuestas automáticas que dará tu agente a ciertas preguntas o comandos.
5. Personaliza la apariencia de tu agente, como su nombre, imagen y colores.
6. Prueba tu agente para asegurarte de que funcione correctamente.
7. Publica tu agente para que esté disponible para interactuar con los usuarios.

¡Listo! Ahora tienes un agente creado con Langgraph que puede ayudar a los usuarios a obtener respuestas a sus preguntas o realizar ciertas tareas de forma automatizada.


# Stream

In [17]:
from langserve import RemoteRunnable
from langchain_core.messages import HumanMessage, AIMessage

remote_runnable = RemoteRunnable("http://localhost:8000/bot/")

In [18]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream(
        input={"messages": [human]},
        config={
            "configurable": {
                "thread_id": "",
                "thread_ts": "string",
                "user_name": "Noel",
            }
        },
    ):
        # Agent Action
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(
                    f"Calling Tool ```{action['tool']}``` with input ```{action['tool_input']}```"
                )
        # Observation
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"Got result: ```{step['observation']}```")
        # Final result
        elif "output" in chunk:
            print(chunk)
            ai = AIMessage(content=chunk["output"])
        else:
            print(chunk)
        print("------")
    chat_history.extend([HumanMessage(content=human), ai])

AI: 
{'assistant': {'messages': AIMessage(content='¡Hola! ¿En qué puedo ayudarte hoy?', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-96ae05fe-7e66-4646-8a5e-3ce8ed665ec7-0')}}
------
AI: 
{'assistant': {'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mCIu7FGSr6nPAYkqWI8X0H9f', 'function': {'arguments': '{}', 'name': 'get_yomap_service_categories'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-71f2e6e6-7df9-4a7a-a670-e18cb52a0ea9-0', tool_calls=[{'name': 'get_yomap_service_categories', 'args': {}, 'id': 'call_mCIu7FGSr6nPAYkqWI8X0H9f'}])}}
------
{'tools': {'messages': [{'content': '["cerrajeria", "piano", "asistente_virtual", "8_56_hierro", "assistant", "tutorias", "muebles", "transporte_de_alimentos", "carrozas", "plomero", "taxi", "tv", "ai", "musica", "depilacion", "llaves_perdidas", "mototaxi", "moda", "asistente", "indriv

In [43]:
def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [44]:
_printed = set()

In [73]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream(
        input={"messages": [human]},
        config={"configurable": {"thread_id": "", "thread_ts": "string"}},
    ):
        if "assistant" in chunk.keys():
            print(chunk["assistant"]["messages"].content)

AI: 
¡Hola! ¿En qué puedo ayudarte hoy?
AI: 


Lo siento, parece que no pude recuperar la información de los dos primeros proveedores de servicios spa. ¿Te gustaría que intente buscar más proveedores de spa para mostrarte su información?
AI: 

Carolina es una artista de manicura premiada que se especializa en diseño y cuidado de uñas. Aquí tienes más información sobre ella:

- Edad: 29
- Género: Femenino
- Promedio de valoraciones: 4.5
- Servicio: Spa
- Dirección: 3602, Nicanor de Obarrio O 4ta Sur, Panamá, Provincia de Panamá, Panamá
- Acerca de ella: Nails design and care. Artista manicura: bodas, shows, películas.
- Redes sociales:
  - [Tiktok](https://www.tiktok.com/@fisicamr/video/7304286339388804385)
  - [Instagram](https://instagram.com/tendencia_unas?igshid=OGQ5ZDc2ODk2ZA==)
  - [Facebook](https://m.facebook.com/spamanospty)

¡Si necesitas más información o ayuda con algo más, házmelo saber!
AI: 


Los servicios para el hogar disponibles en YoMap incluyen proveedores como Akshd

In [67]:
chunk["assistant"]["messages"].content

'Akshdeep is a 15-year-old male service provider located in Sudan, Punjab, India. He specializes in home services. If you need assistance with home-related tasks, Akshdeep may be able to help.'